## Name: Samujjal Seal Sarma            

## Text classification using Python:

## Approach:
1. Importing packages.
2. Scraping data from wikipedia various topics as mentioned below using Pythons beautiful soup and create a data frame of various documents.
    - Cricket
    - Movie
    - Technology
    - Health
3. Cleaning the data by removing stop words and special characters using regex and then lemmetizing the words.
4. Creating a document token matrix.
5. Using ML algorithms like Logistic regression and random forest for test classification and prediction. Got a prediction of approx 99% through radom forest.

### Importing necessary packages

In [1]:
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import requests
import bs4
import pandas as pd
import re
from sklearn.decomposition import LatentDirichletAllocation,PCA
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# Getting rid of unnecessary warnings
import warnings; 
warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\samuj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\samuj\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\samuj\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Steps to scrape data from wikipedia
    1. Create URL links for each category: 
        - Cricket
        - Movie
        - Technology
        - Health
    2. Build a Dataframe constituting 500 paragraphs for each of the above categories
        - In dataframe, create 2 columns: Paragraph and category
        - Build dataframe of size 2000 (500 paragraphs for each category)


In [2]:
#Cricket URLs
cric_url = 'https://en.wikipedia.org/wiki/Cricket'
t20_url = 'https://en.wikipedia.org/wiki/Twenty20'
test_url = 'https://en.wikipedia.org/wiki/Test_cricket'
wc_url = 'https://en.wikipedia.org/wiki/Cricket_World_Cup'
ipl_url = 'https://en.wikipedia.org/wiki/Indian_Premier_League'
t20wc_url = 'https://en.wikipedia.org/wiki/ICC_T20_World_Cup'
cricindia_url = 'https://en.wikipedia.org/wiki/Cricket_in_India'
indiacricteam_url = 'https://en.wikipedia.org/wiki/India_national_cricket_team'
histworldcup_url = 'https://en.wikipedia.org/wiki/History_of_the_ICC_Cricket_World_Cup'
champs_url = 'https://en.wikipedia.org/wiki/Champions_League_Twenty20'
engcric_url = 'https://en.wikipedia.org/wiki/Cricket#English_cricket_in_the_18th_and_19th_centuries'
ltdoverscric_url = 'https://en.wikipedia.org/wiki/Limited_overs_cricket'

cricket_urls = [cric_url,  t20_url, test_url, wc_url, ipl_url, t20wc_url, cricindia_url, indiacricteam_url, champs_url,
               engcric_url,ltdoverscric_url]
#Film URLs
film_url = 'https://en.wikipedia.org/wiki/Film'
bolly_url = 'https://en.wikipedia.org/wiki/Bollywood'
cineindia_url = 'https://en.wikipedia.org/wiki/Cinema_of_India'
holly_url = 'https://en.wikipedia.org/wiki/Hollywood'
cineus_url = 'https://en.wikipedia.org/wiki/Cinema_of_the_United_States'
filmhist_url = 'https://en.wikipedia.org/wiki/History_of_film'

film_urls = [film_url, bolly_url, cineindia_url, holly_url, cineus_url, filmhist_url]

#Tech URLs
tech_url = 'https://en.wikipedia.org/wiki/Technology'
techroad_url = 'https://en.wikipedia.org/wiki/Technology_roadmap'
techsociety_url = 'https://en.wikipedia.org/wiki/Technology_and_society'
techushist_url = 'https://en.wikipedia.org/wiki/Technological_and_industrial_history_of_the_United_States'
techanalysis_url = 'https://en.wikipedia.org/wiki/Technical_analysis'
techwar_url = 'https://en.wikipedia.org/wiki/Technology_during_World_War_I'
techconv_url = 'https://en.wikipedia.org/wiki/Technological_convergence'
techit_url = 'https://en.wikipedia.org/wiki/Information_technology'
techcomp_url = 'https://en.wikipedia.org/wiki/Computer'
techcompport_url='https://en.wikipedia.org/wiki/Portable_computer'

tech_urls = [tech_url, techroad_url, techsociety_url, techushist_url, techanalysis_url, techwar_url, techconv_url,
            techit_url, techcomp_url, techcompport_url]


#Health URLs
health_url = 'https://en.wikipedia.org/wiki/Health'
healthus_url = 'https://en.wikipedia.org/wiki/Health_care_in_the_United_States'
healthins_url = 'https://en.wikipedia.org/wiki/Health_insurance'
healthcare_url = 'https://en.wikipedia.org/wiki/Health_care'
healthcan_url = 'https://en.wikipedia.org/wiki/Healthcare_in_Canada'
healthport_url = 'https://en.wikipedia.org/wiki/Health_Insurance_Portability_and_Accountability_Act'
healthtob_url = 'https://en.wikipedia.org/wiki/Health_effects_of_tobacco'

health_urls = [health_url, healthus_url, healthins_url, healthcare_url, healthcan_url,healthport_url, healthtob_url ]

all_categ_urls = [cricket_urls, film_urls, tech_urls, health_urls ]
topics = ["Cricket","Films","Tech","Health"]
iTopic = 0
df = pd.DataFrame(columns = ['Paragraph','Category'])

#Preparing dataframe out of urls paragraph data
topic_limit = 500
i = 0
for categ_url in all_categ_urls:
    topic_ctr = 0
    limit_reached = 0
    topic = topics[iTopic]
    iTopic += 1
    for url in categ_url:
            if (limit_reached == 1):
                break
            r = requests.get(url)
            soup = BeautifulSoup(r.content, 'html.parser')
            table = soup.find_all('p',attrs={'class': None})
            for x in table:
                if(len(str(x.get_text().strip())) > 0):
                    topic_ctr = topic_ctr + 1
                    df.loc[i] = [str(x.get_text()),topic]
                    i = i + 1
                    if (topic_ctr >= topic_limit ):
                        limit_reached = 1
                        break
print("Succesfully created dataframe of size: ",len(df))
print("Listing Top few rows: ")
df.head()

Succesfully created dataframe of size:  2000
Listing Top few rows: 


,Paragraph,Category
0,Cricket is a bat-and-ball game played between ...,Cricket
1,There are various formats ranging from Twenty2...,Cricket
2,"Historically, cricket's origins are uncertain ...",Cricket
3,"Cricket is one of many games in the ""club ball...",Cricket
4,It is generally believed that cricket originat...,Cricket


### Steps performed in below code:
1. Cleaning:
    - Special characters (e.g. !,@, #)
    - Single characters (e.g. a, i)
    - Numbers
    - Multiple spaces and prefixes
2. Lemmatization
3. Shuffling

In [3]:
#Defining function for cleaning text
def cleanText(text):
    '''text = re.sub('[^ a-zA-Z]','',text)
    text = re.sub(r' +', ' ', text)'''
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(text))
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)     
    # Remove numbers characters from the start
    document = re.sub(r'[0-9]', ' ', document)
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    # Converting to Lowercase
    document = document.lower()
    return text

#Defining function for lemmatization
wnl = WordNetLemmatizer()
def lemmatizeSentences(data):
    count = 0    
    for text in data["Paragraph"]:
        # clean the text
        text = cleanText(text)
        sentence = ""
        for i, j in pos_tag(word_tokenize(text)):
            word = ''
            if j[0].lower() in ['n', 'v', 'r']:
                word = wnl.lemmatize(i, j[0].lower())
            elif j[0].lower() is 'j':
                word = wnl.lemmatize(i, 'a')
            else:
                word = wnl.lemmatize(i)
            sentence = sentence + " " + word.lower()
        data["Paragraph"].iloc[count] = sentence
        count = count + 1;
    return data
df = lemmatizeSentences(df)
#Shuffling the data
df = df.sample(frac=1).reset_index(drop=True)
print("Completed cleaning process successfully for dataframe of size: ", len(df))
print("Listing top few rows after cleaning:")
df.head()

Completed cleaning process successfully for dataframe of size:  2000
Listing top few rows after cleaning:


,Paragraph,Category
0,"as the stalemate develop on the ground , with...",Tech
1,numerous canal company have also be charter ;...,Tech
2,healthcare in canada be deliver through thirt...,Health
3,"the importance of stone tool , circa 2.5 mill...",Tech
4,an example of a security that have an apparen...,Tech


### Steps performed in below code:
1. DTM creation:
    - Each column of dtm will contain number of times each word appears in each category    

In [4]:
#Creating dtm
vectorizer = CountVectorizer(min_df = 0.05 , max_df= 0.95, stop_words="english")
dtm = pd.DataFrame(vectorizer.fit_transform(df['Paragraph']).toarray(),columns=vectorizer.get_feature_names())
dtm.head()

,australia,begin,bollywood,care,century,change,cinema,company,computer,cost,...,term,test,time,united,use,war,win,work,world,year
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,3,0,0,0,0,0,0,0


### Steps performed in below code:
1. Logistic Regression Algorithm:
    - DTM from above code is split into training data (70%) and test data (30%)
    - Logistic regression algo is run
    - Using test data, model is trained again, re-run the algo to find change in accuracy

In [5]:
#Logistic regression
X_train, X_test, y_train, y_test = train_test_split(dtm,df["Category"],test_size=0.30)
logisticModel = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')
logisticModel.fit(X_train,y_train)
logisticModel.score(X_test,y_test)

0.8983333333333333

In [6]:
cross_val_score(logisticModel,dtm, df['Category'].astype('str'),cv=4).mean()

0.8995

### Steps performed in below code:
1. Random Forest regression algorithm:
    - DTM from above code is split into training data (70%) and test data (30%)
    - Random Forest regression algo is executed
    - Using test data, model is trained again, re-run the algo to find change in accuracy

In [7]:
#get dtm columns to be used as training and test
columnList = list(dtm.columns.values)
trainingColumns = int(len(columnList)-1)
X = columnList[:trainingColumns]
y = columnList[:len(columnList)-trainingColumns]

In [8]:
#RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
X_train1, X_test1, y_train1, y_test1 = train_test_split(dtm,dtm[y],test_size=0.30)
randomModel = RandomForestRegressor(n_estimators=100, random_state=42)  
randomModel.fit(X_train1, y_train1) 
randomModel.score(X_test1,y_test1)

0.9989549614861805